In [9]:
from __future__ import division
from mnist import MNIST
from sklearn import tree
import numpy as np
from sklearn.metrics import classification_report,log_loss,accuracy_score,roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from time import time
import pandas as pd
from datetime import datetime
from openpyxl import load_workbook
import re
from sklearn.preprocessing import label_binarize
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from mlxtend.data import loadlocal_mnist
import timeit
import timeit 
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import confusion_matrix


images_train_o,labels_train_o  = loadlocal_mnist(
        images_path='/users/mtech/jatindev/Downloads/train-images.idx3-ubyte',
        labels_path='/users/mtech/jatindev/Downloads/train-labels.idx1-ubyte')
images_test_o,labels_test_o= loadlocal_mnist(
        images_path='/users/mtech/jatindev/Downloads/t10k-images.idx3-ubyte',
        labels_path='/users/mtech/jatindev/Downloads/t10k-labels.idx1-ubyte')



In [10]:
excel_file_name = 'landmarksvm' + str(datetime.now().strftime('%Y%m%d%H%M%S')) + '.xlsx'
excel_writer = pd.ExcelWriter(excel_file_name, engine='openpyxl')
df = pd.DataFrame(columns=['gamma','landmarks','Examples','Accuracy','Normalized Accuracy', 
                     'Time_taken_train','Time_taken_test',
                     'Negative Log loss','Macro_auc','Micro_auc'])
df.to_excel(excel_writer, sheet_name='demo_sheet')

excel_writer.save()


/usr/lib/python3/dist-packages/pandas/io/excel.py:687: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  self.book.remove_sheet(self.book.worksheets[0])


In [11]:
def classifaction_report_to_dataframe(report):
    report_data = []
    lines = report.split('\n')
    for line in lines[2:-3]:
        row = {}
        row_data = line.split('      ')
        print(line)
        if len(row_data)>5:
            row['class'] = row_data[1]
            row['precision'] = float(row_data[2])
            row['recall'] = float(row_data[3])
            row['f1_score'] = float(row_data[4])
            row['support'] = float(row_data[5])
            report_data.append(row)
    for line in lines[-2:-1]:
        row = {}
        p = re.compile("weighted avg")
        m = p.search(line)  
        print(line)
        row['class'] = m.group(0)
        p = re.compile("\d+.?\d+")
        row_data = p.findall(line)
        row['precision'] = float(row_data[0])
        row['recall'] = float(row_data[1])
        row['f1_score'] = float(row_data[2])
        row['support'] = float(row_data[3])
        report_data.append(row)
        
    dataframe = pd.DataFrame.from_dict(report_data)
    return dataframe

In [ ]:
for gamma in[0.01,0.0001,0.0000001,0.0000000001]:
    for l in [1,2,5,50,100,250,784]:
        s=np.random.randint(l, size=60000)
        print(gamma,l)
        images_train=images_train_o
        labels_train=labels_train_o
        images_test=images_test_o
        labels_test=labels_test_o
        Trans_Train_data=np.zeros((60000,l))
        Trans_Test_data=np.zeros((10000,l))
        for i in range(0,60000):
            for j in range(0,l):
               Trans_Train_data[i][j]  =  np.exp(-gamma *(np.linalg.norm(images_train[i]-images_train[s[j]])) **2 )

        for i in range(0,10000):
            for j in range(0,l):
                Trans_Test_data[i][j]=np.exp( -gamma*(np.linalg.norm(images_test[i]-images_train[s[j]])) **2 )    
        images_train=Trans_Train_data
        images_test=Trans_Test_data
        svm = LinearSVC(dual=True)
        classifier= CalibratedClassifierCV(svm)
        start=time()
        classifier.fit(images_train,labels_train)
        train_time=time()-start
        start=time()
        predictions=classifier.predict(images_test)
        test_time=time()-start
        accuracy=accuracy_score(predictions,labels_test,normalize=False)
        normalized_accuracy=accuracy_score(predictions,labels_test,normalize=True)
        log_loss_prediction=classifier.predict_proba(images_test)
        log_loss_val=log_loss(labels_test,log_loss_prediction)
        classification_repo=classification_report(labels_test,predictions)

        labels_test_one_hot=label_binarize(labels_test,classes=[0,1,2,3,4,5,6,7,8,9])
        predictions_one_hot=label_binarize(predictions,classes=[0,1,2,3,4,5,6,7,8,9])
        micro_auc=roc_auc_score(labels_test_one_hot,predictions_one_hot,average="micro")
        macro_auc=roc_auc_score(labels_test_one_hot,predictions_one_hot,average="macro")

        #print(micro_auc,macro_auc)
        cm = confusion_matrix(labels_test,predictions)
        cm_df = pd.DataFrame(data=cm)
        df_data = [gamma,l,60000,accuracy,normalized_accuracy,train_time,test_time,log_loss_val,macro_auc,micro_auc]
        df.loc[len(df)] = df_data   

        book = load_workbook(excel_file_name)
        with pd.ExcelWriter(excel_file_name, engine='openpyxl') as excel_writer:
            excel_writer.book = book
            df_classification_report = classifaction_report_to_dataframe(classification_repo)
            df_classification_report.to_excel(excel_writer, sheet_name='Landmarksvm_cr'+str(60000))

            cm_df.to_excel(excel_writer, sheet_name='Landmarksvm_cr'+str(60000),
                           startrow=len(df_classification_report)+5)

            excel_writer.save()

book = load_workbook(excel_file_name)
with pd.ExcelWriter(excel_file_name, engine='openpyxl') as excel_writer:
    excel_writer.book = book
    df.to_excel(excel_writer, sheet_name='Landmarksvm')
    excel_writer.save()



0.01 1


/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/lib/python3/dist-packages/pandas/io/excel.py:687: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  self.book.remove_sheet(self.book.worksheets[0])


           0       0.00      0.00      0.00       980
           1       0.11      1.00      0.20      1135
           2       0.00      0.00      0.00      1032
           3       0.00      0.00      0.00      1010
           4       0.00      0.00      0.00       982
           5       0.00      0.00      0.00       892
           6       0.00      0.00      0.00       958
           7       0.00      0.00      0.00      1028
           8       0.00      0.00      0.00       974
           9       0.00      0.00      0.00      1009

   micro avg       0.11      0.11      0.11     10000
weighted avg       0.01      0.11      0.02     10000
0.01 2


/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/lib/python3/dist-packages/pandas/io/excel.py:687: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  self.book.remove_sheet(self.book.worksheets[0])


           0       0.00      0.00      0.00       980
           1       0.11      1.00      0.20      1135
           2       0.00      0.00      0.00      1032
           3       0.00      0.00      0.00      1010
           4       0.00      0.00      0.00       982
           5       0.00      0.00      0.00       892
           6       0.00      0.00      0.00       958
           7       0.00      0.00      0.00      1028
           8       0.00      0.00      0.00       974
           9       0.00      0.00      0.00      1009

   micro avg       0.11      0.11      0.11     10000
weighted avg       0.01      0.11      0.02     10000
0.01 5


/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/users/mtech/jatindev/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/lib/python3/dist-packages/pandas/io/excel.py:687: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  self.book.remove_sheet(self.book.worksheets[0])


           0       0.00      0.00      0.00       980
           1       0.11      1.00      0.20      1135
           2       0.00      0.00      0.00      1032
           3       0.00      0.00      0.00      1010
           4       0.00      0.00      0.00       982
           5       0.00      0.00      0.00       892
           6       0.00      0.00      0.00       958
           7       0.00      0.00      0.00      1028
           8       0.00      0.00      0.00       974
           9       0.00      0.00      0.00      1009

   micro avg       0.11      0.11      0.11     10000
weighted avg       0.01      0.11      0.02     10000
0.01 50
